# Image Basics

In [36]:
# imports
import os
from pkg_resources import resource_filename
import glob

import subprocess

import numpy as np

from astropy.io import fits

# ASTR 257
from astr257 import img_utils

# Image lists

## Generate a dummy folder for this Notebook

In [35]:
dummy_name = 'Dummy_folder'
if not os.path.isdir(dummy_name):
    os.mkdir(dummy_name)

## Generate dummy "images" (the names are all that matter for what follows,)

In [38]:
for ii in range(100):
    subprocess.Popen(['touch', dummy_name+'/dummy{}.fits'.format(ii)])

## Load em

In [40]:
dum_files = glob.glob(dummy_name+'/dummy*fits')
dum_files[0:5]

['Dummy_folder/dummy25.fits',
 'Dummy_folder/dummy64.fits',
 'Dummy_folder/dummy99.fits',
 'Dummy_folder/dummy31.fits',
 'Dummy_folder/dummy29.fits']

In [41]:
dum_files.sort()
dum_files[0:5]

['Dummy_folder/dummy0.fits',
 'Dummy_folder/dummy1.fits',
 'Dummy_folder/dummy10.fits',
 'Dummy_folder/dummy11.fits',
 'Dummy_folder/dummy12.fits']

## Deal with funny sorting

In [44]:
all_raw1 = glob.glob(os.path.join(dummy_name, 'dummy?.fits'))
all_raw2 = glob.glob(os.path.join(dummy_name, 'dummy??.fits'))
all_raw3 = glob.glob(os.path.join(dummy_name, 'dummy???.fits'))
all_raw1.sort()
all_raw2.sort()
all_raw3.sort()

all_raw = all_raw1 + all_raw2 + all_raw3

In [45]:
all_raw[0:5]

['Dummy_folder/dummy0.fits',
 'Dummy_folder/dummy1.fits',
 'Dummy_folder/dummy2.fits',
 'Dummy_folder/dummy3.fits',
 'Dummy_folder/dummy4.fits']

# Load an image

In [4]:
# Test image from our Repo
tst_file = os.path.join(resource_filename('astr257', 'data'), 'images', 'd23.fits.gz')

In [5]:
os.path.isfile(tst_file)

True

## Load into an HDU List

In [7]:
hdul = fits.open(tst_file)

### Inspect the HDU List

In [9]:
hdul.info()

Filename: /home/xavier/Classes/ASTR257/astr257/data/images/d23.fits.gz
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU     153   (1056, 1024)   int16 (rescales to uint16)   


### Load the Header

In [10]:
head0 = hdul[0].header

In [11]:
# Inspect
head0

SIMPLE  =                    T / NORMAL FITS IMAGE                              
BITPIX  =                   16 / DATA PRECISION                                 
NAXIS   =                    2 / NUMBER OF IMAGE DIMENSIONS                     
NAXIS1  =                 1056 / NUMBER OF COLUMNS                              
NAXIS2  =                 1024 / NUMBER OF ROWS                                 
CRVAL1U =                 2048 / COLUMN ORIGIN                                  
CRVAL2U =                 2048 / ROW ORIGIN                                     
CDELT1U =                   -2 / COLUMN CHANGE PER PIXEL                        
CDELT2U =                   -2 / ROW CHANGE PER PIXEL                           
OBSNUM  =                   23 / OBSERVATION NUMBER                             
IDNUM   =                    7 / IMAGE ID                                       
UGEOM   =                    0 / UCAM READOUT GEOMETRY                          
DGEOM   =                   

In [18]:
# This object is dict-like
head0['NAXIS1']

1056

In [20]:
# And malleable (so be careful with it)
head0['NAXIS2'] = 999
head0['NAXIS2']

999

## Load the data

In [12]:
data = hdul[0].data

In [14]:
data  # Note, these are unit16!  Python can do things with these you might not expect

array([[1453, 1356, 1396, ..., 1130, 1133, 1128],
       [2033, 1642, 1672, ..., 1134, 1140, 1138],
       [2210, 1667, 1659, ..., 1142, 1134, 1133],
       ...,
       [1623, 1359, 1362, ..., 1150, 1155, 1156],
       [1568, 1350, 1332, ..., 1156, 1155, 1157],
       [1645, 1441, 1408, ..., 1152, 1161, 1151]], dtype=uint16)

In [16]:
### Convert to float
data = data.astype(float)  # float64, which is excessive but standard for Python
data

array([[1453., 1356., 1396., ..., 1130., 1133., 1128.],
       [2033., 1642., 1672., ..., 1134., 1140., 1138.],
       [2210., 1667., 1659., ..., 1142., 1134., 1133.],
       ...,
       [1623., 1359., 1362., ..., 1150., 1155., 1156.],
       [1568., 1350., 1332., ..., 1156., 1155., 1157.],
       [1645., 1441., 1408., ..., 1152., 1161., 1151.]])

----

# View an Image (with Ginga)

## Need to launch the Ginga window first, from your terminal
    ginga --modules=RC

In [17]:
# Now a handy method we provide
img_utils.view_in_ginga(data)

----

# Write a new image

In [21]:
data2 = data*10.

## Generate a Primary HDU
    # This are frequently 'empty' but ours will contain the image

In [27]:
phdu = fits.PrimaryHDU(data2)
phdu.header

SIMPLE  =                    T / conforms to FITS standard                      
BITPIX  =                  -64 / array data type                                
NAXIS   =                    2 / number of array dimensions                     
NAXIS1  =                 1056                                                  
NAXIS2  =                 1024                                                  
EXTEND  =                    T                                                  

## Generate an HDUList

In [29]:
hdul2 = fits.HDUList([phdu])

## Write to FITS

In [31]:
hdul2.writeto('tmp.fits', overwrite=True)